This notebook groups journeys by the starting and ending area (residential, non-residential, mixed etc.). The function takes CSVs, of which there is one for every week, and groups the journeys by the starting and ending area.

# Import libraries and data

In [6]:
import pandas as pd
from datetime import datetime
from geopy import distance

In [7]:
bike_locations_import = pd.read_csv("bike_locations_extra.csv")
bike_locations_import.head()


,Dock_ID,commonName_x,latitude_x,longitude_x,NbDocks_x,borough_x,postcode_x,description_code_x,oa11,laua,ward,lsoa11,msoa11,oac11,elevation,nearest_bus,nearest_tube,nearest_rail
0,684,"Neville Gill Close, Wandsworth",51.454752,-0.195197,32,Wandsworth,SW18 4TH,U,E00023152,E09000032,E05000624,E01004600,E02000940,3B1,6.285259,137.955060,1251.650607,917.897377
1,704,"Mexfield Road, East Putney",51.456820,-0.202802,18,Wandsworth,SW15 2RQ,R,E00022723,E09000032,E05000614,E01004511,E02000932,2D2,18.794371,109.129073,718.838716,1114.147350
2,767,"Santos Road, Wandsworth",51.457059,-0.200806,30,Wandsworth,SW18 1NS,R,E00022696,E09000032,E05000614,E01004511,E02000932,2D2,15.571350,21.880924,801.284683,938.622567
3,766,"Ram Street, Wandsworth",51.457529,-0.192165,25,Wandsworth,SW18 4LD,M,E00022687,E09000032,E05000614,E01004509,E02000932,3B3,6.049758,51.354330,1340.192303,524.553329
4,637,"Spencer Park, Wandsworth Common",51.457870,-0.174691,25,Wandsworth,SW18 2SR,R,E00022718,E09000032,E05000614,E01004508,E02000934,2D2,22.749346,148.450561,1894.828615,840.241706


# Function to export grouped data for exploration in Tableau

In [8]:
#function to clean and reduce the number of categories describing the areas. 
#Non-residential, partly non-residential and large user all become non-residential
# Residential and partly residential become residential and mixed areas are left as they are.
def clean_descriptions(x):
    description = ""
    if x in ["NR", "PNR", "LU"]:
        description = "NR"
    elif x in ["R", "PR"]:
        description = "R"
    else:
        description = x
    return description    

In [10]:
all_weeks=pd.DataFrame(columns=['MM', 'MNR', 'MR', 'NRM', 'NRNR', 'NRR', 'RM', 'RNR', 'RR'])

In [41]:
for i in range(143,253):
    print(i)
    trip_data_import = pd.read_csv("C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/All bike wweeks/"+str(i)+".csv")
    trip_data_import.head()
    merged_data = trip_data_import.merge(bike_locations_import, 
                left_on='StartStation Id', right_on="Dock_ID", how='left').merge(bike_locations_import, 
                left_on="EndStation Id", right_on="Dock_ID",
                how='left',suffixes=('_start', '_end'))

    merged_data = merged_data[["Dock_ID_end", "Dock_ID_start","Start Date", "End Date","Duration","latitude_x_start", "longitude_x_start",
                              "latitude_x_end", "longitude_x_end", 
                              "description_code_x_start", 'description_code_x_end']]

    merged_data.columns = ["start_dock", "end_dock","start_date", "end_date","duration", "lat_start", "lon_start", 
                           "lat_end", "lon_end", "area_start", "area_end"]



    merged_data["area_start"]=merged_data["area_start"].apply(clean_descriptions)
    merged_data["area_end"]=merged_data["area_end"].apply(clean_descriptions)

    merged_data=merged_data[merged_data["area_start"]!="U"]
    merged_data=merged_data[merged_data["area_end"]!="U"]
  
    merged_data = merged_data.dropna()
    merged_data = merged_data.reset_index(drop=True)
    
    merged_data.to_csv("merged_data_"+str(i)+".csv")
    cluster_data = merged_data[["area_start", "area_end"]]
    df=cluster_data.groupby(by=["area_start", "area_end"]).size().reset_index()
    df.columns=["area_start", "area_end", "count"+str(i)]
    df["area_comb"]=df["area_start"]+df["area_end"]
    df=df.set_index('area_comb')
    all_weeks=all_weeks.append(df["count"+str(i)])


143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252


In [49]:
#export the data for Tableau
all_weeks.to_csv("all_weeks_clusters.csv")

In [53]:
#a function which returns the distance made on each journey
from geopy import distance
for j in range(143,253):
    data_import=pd.read_csv("merged_data_"+str(j)+".csv")
    data_import["distance"]=""
    for i in range(data_import.shape[0]):
        try:
            start = (data_import.at[i, "lat_start"],data_import.at[i, "lon_start"])
            end = (data_import.at[i, "lat_end"],data_import.at[i, "lon_end"])
            data_import.at[i,"distance"]=distance.distance(start, end).km
        except ValueError:
            print (i)
            continue
    data_import.to_csv(str(j)+"_distance.csv")